In [1]:
!pip install vaderSentiment

In [53]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
intensity = SentimentIntensityAnalyzer()

intensity.polarity_scores("This food is GOOD, service is not GOOD")

{'neg': 0.0, 'neu': 0.452, 'pos': 0.548, 'compound': 0.8056}

In [3]:
data = pd.read_excel('D:\\Un structured data\\Amazon.xlsx')


text = data[['reviews.text']]
text.head()

,reviews.text
0,I initially had trouble deciding between the p...
1,Allow me to preface this with a little history...
2,I am enjoying it so far. Great for reading. Ha...
3,I bought one of the first Paperwhites and have...
4,I have to say upfront - I don't like coroporat...


In [4]:
text['clean_text'] = text['reviews.text'].str.replace('[^a-zA-Z ]','')

text.head()

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,reviews.text,clean_text
0,I initially had trouble deciding between the p...,I initially had trouble deciding between the p...
1,Allow me to preface this with a little history...,Allow me to preface this with a little history...
2,I am enjoying it so far. Great for reading. Ha...,I am enjoying it so far Great for reading Had ...
3,I bought one of the first Paperwhites and have...,I bought one of the first Paperwhites and have...
4,I have to say upfront - I don't like coroporat...,I have to say upfront I dont like coroporate ...


In [5]:
#creating a UDF to apply on sentiment

text_polarity = lambda x:intensity.polarity_scores(x)['compound']

In [6]:
text['polarity_score'] = text['clean_text'].apply(text_polarity)
text.head()

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,reviews.text,clean_text,polarity_score
0,I initially had trouble deciding between the p...,I initially had trouble deciding between the p...,0.9799
1,Allow me to preface this with a little history...,Allow me to preface this with a little history...,0.9868
2,I am enjoying it so far. Great for reading. Ha...,I am enjoying it so far Great for reading Had ...,0.4364
3,I bought one of the first Paperwhites and have...,I bought one of the first Paperwhites and have...,0.9734
4,I have to say upfront - I don't like coroporat...,I have to say upfront I dont like coroporate ...,0.9922


In [7]:
def pola(ps):

    if ps >=.7:
        return 'Highly Positive'
    elif ps<.7 and ps>.35:
        return 'Positive'
    elif ps <=.35 and ps>=-.35:
        return 'Neutral'
    elif ps <-.35 and ps>=-.7:
        return 'Negative'
    else:
        return 'Highly Negative'


In [8]:
text['sentiment'] = text['polarity_score'].apply(pola)

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
text.head()

,reviews.text,clean_text,polarity_score,sentiment
0,I initially had trouble deciding between the p...,I initially had trouble deciding between the p...,0.9799,Highly Positive
1,Allow me to preface this with a little history...,Allow me to preface this with a little history...,0.9868,Highly Positive
2,I am enjoying it so far. Great for reading. Ha...,I am enjoying it so far Great for reading Had ...,0.4364,Positive
3,I bought one of the first Paperwhites and have...,I bought one of the first Paperwhites and have...,0.9734,Highly Positive
4,I have to say upfront - I don't like coroporat...,I have to say upfront I dont like coroporate ...,0.9922,Highly Positive


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [41]:
x = text['clean_text']
y = text['sentiment']

In [43]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [44]:
tfid_vectorizer = TfidfVectorizer(stop_words='english',encoding='ISO-8859-1')

In [46]:
tfid_vectorizer.fit(x_train)
tfidf_train = tfid_vectorizer.transform(x_train)
tfidf_test = tfid_vectorizer.transform(x_test)

# Multinomial

In [50]:
nav = MultinomialNB()
nav.fit(tfidf_train,y_train)

prediction = nav.predict(tfidf_test)
accuracy_c_v = metrics.accuracy_score(y_test,prediction)
print(accuracy_c_v)

0.7375


# Passive Aggressive

In [54]:
from sklearn.linear_model import PassiveAggressiveClassifier

pasive = PassiveAggressiveClassifier()
pasive.fit(tfidf_train,y_train)

prediction = pasive.predict(tfidf_test)
accuracy = metrics.accuracy_score(y_test,prediction)
print(accuracy)

0.778125


# Random Forest

In [78]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(tfidf_train,y_train)

prediction = rf.predict(tfidf_test)
accuracy = metrics.accuracy_score(y_test,prediction)
print(accuracy)

0.8125
